In [40]:
import numpy as np
import pandas as pd
import os
from scipy import signal
from math import floor,inf
from sklearn.utils import shuffle
from mne.time_frequency import psd_array_multitaper
from scipy.integrate import simps

In [41]:
##Mi task labels mapped to numbers
def map_labels(label):
    if label == 'Foot':
        label = 0
    if label =='Left Hand':
        label = 1
    if label == 'Right Hand':
        label = 2
    if label == 'Tongue':
        label = 3
    return label

In [42]:
###Function to process raw EEG into filtered 2s segments and create training and validation datasets 
def create_data(data_folder, filtered, samples, overlap, classes):
    b,a = signal.butter(2, Wn = [48,52], btype = 'bandstop', fs = 250)
    d,c = signal.butter(2, Wn = [4,38], btype = 'bandpass', fs = 250)
    data_dir = os.path.join(os.getcwd(),data_folder)
    
    mi_tasks=classes

    directories = {}
    for i in range(len(mi_tasks)):
        directories[mi_tasks[i]] = data_dir + '\\' + mi_tasks[i]

    samples = samples
    overlap = overlap
    unique = floor((1-overlap)*samples)
    x=[]
    y=[]
    all_data = {}
    filtered_data = {}

    for task in mi_tasks:
        all_data[task] = {}
        filtered_data[task] = {}    

    for i in directories:
        folders = os.listdir(directories[i])
        count = 0
        for w in range(len(folders)):
            current_dir = directories[i] + '\\' + folders[w]
            files = os.listdir(current_dir)
            for file in files:
                data = pd.read_csv(current_dir + '\\' + file, header = None)
                eeg_data = data.iloc[1:,1:9]
                if eeg_data.size >= 3250: 
                    for div in range(int(eeg_data.size//(eeg_data.shape[1]*unique))):
                        if div*unique +samples <= eeg_data.size/eeg_data.shape[1]:
                            all_data[i][count]= eeg_data[div*unique:div*unique + samples]
                            if filtered:
                                to_filter = all_data[i][count]
                                filtered_data=pd.DataFrame(columns = range(0,eeg_data.shape[1]))
                                for chan in range(to_filter.shape[1]):
                                    notch_filtered_chan = signal.filtfilt(b,a,to_filter.iloc[:,chan], padtype='even')
                                    filtered_chan = signal.filtfilt(d,c,notch_filtered_chan, padtype='even')
                                    filtered_data[chan] = filtered_chan
                                x.append(np.transpose(filtered_data.to_numpy()))
                                    
                            else:
                                x.append(np.transpose(all_data[i][count].to_numpy()))
                            
                            y.append(i)
                            count += 1

                else:
                    print('* not accepted *')
                    print(file)
    x=np.array(x)
    #x=x.reshape(x.shape[0],x.shape[1],x.shape[2],1)
    y = list(map(map_labels,y))
    #x, y = shuffle(x,y, random_state=42)
    return all_data, x, y

In [43]:
train_data_folder = '05to20-08 Active 8chan FPZ Bias'
test_data_folder = '23&24-08 Active 8chan FPZ Bias'

In [44]:
train_data, x_train, y_train = create_data(train_data_folder, True, 500, 0, ['Foot', 'Left Hand', 'Right Hand', 'Tongue'])
test_data, x_test, y_test = create_data(test_data_folder, True, 500, 0, ['Foot', 'Left Hand', 'Right Hand', 'Tongue'])
print(x_train.shape)
print(x_test.shape)

(1154, 8, 500)
(384, 8, 500)


In [45]:
####Function to extract only the data from a single electrode for each MI task
def extract_chan_data(x,y,chan):
    start = 0
    sec_data = {}
    for i in np.unique(y):    #classes
        end=y.count(i) + start
        sec_data[i] = []
        for a in range(start,end):
            sec_data[i].append(x[a][chan])
        start=end
    return sec_data
    

In [46]:
####Function to calculate average bandpowers for a specified band (low = lower limit, high = upper limit)
def bandpower_calc(chan_data, low, high):
    bandpowers=[]
    for trial in chan_data:
        psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
        idx = np.logical_and(freqs >= low, freqs <= high)
        freq_res = freqs[1] - freqs[0]
        power = simps(psd[idx], dx=freq_res)
        bandpowers.append(power) 
    return bandpowers

In [ ]:
####### Alpha+Beta (8-26Hz) bandpower calculations for c3,c4,cp1,cp2 (chans 3,4,5,6) ######

In [47]:
####extract train and validation data for each channel

chan_3_train=extract_chan_data(x_train,y_train, 2)
chan_4_train=extract_chan_data(x_train,y_train, 3)
chan_5_train=extract_chan_data(x_train,y_train, 4)
chan_6_train=extract_chan_data(x_train,y_train, 5)

chan_3_test=extract_chan_data(x_test,y_test, 2)
chan_4_test=extract_chan_data(x_test,y_test, 3)
chan_5_test=extract_chan_data(x_test,y_test, 4)
chan_6_test=extract_chan_data(x_test,y_test, 5)



In [48]:
####foot MI

ft_chan3 = bandpower_calc(chan_3_train[0],8,26) 
ft_chan4 = bandpower_calc(chan_4_train[0],8,26)
ft_chan5 = bandpower_calc(chan_5_train[0],8,26)
ft_chan6 = bandpower_calc(chan_6_train[0],8,26)

ft_chan3_test = bandpower_calc(chan_3_test[0],8,26)
ft_chan4_test = bandpower_calc(chan_4_test[0],8,26)
ft_chan5_test = bandpower_calc(chan_5_test[0],8,26)
ft_chan6_test = bandpower_calc(chan_6_test[0],8,26)

<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', v

<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', v

<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', v

<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', v

<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', v

In [49]:
print(np.mean(ft_chan3))
print(np.mean(ft_chan4))
print(np.mean(ft_chan5))
print(np.mean(ft_chan6))
print('\n')
print(np.mean(ft_chan3_test))
print(np.mean(ft_chan4_test))
print(np.mean(ft_chan5_test))
print(np.mean(ft_chan6_test))

5.806578712731838
6.561464655995163
5.822214354539752
6.429513769480121


5.460674202300374
6.802970977902283
5.5592463660582965
11.966921643839123


In [51]:
####LH MI

lh_chan3 = bandpower_calc(chan_3_train[1],8,26)
lh_chan4 = bandpower_calc(chan_4_train[1],8,26)
lh_chan5 = bandpower_calc(chan_5_train[1],8,26)
lh_chan6 = bandpower_calc(chan_6_train[1],8,26)

lh_chan3_test = bandpower_calc(chan_3_test[1],8,26)
lh_chan4_test = bandpower_calc(chan_4_test[1],8,26)
lh_chan5_test = bandpower_calc(chan_5_test[1],8,26)
lh_chan6_test = bandpower_calc(chan_6_test[1],8,26)


<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', v

<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', v

<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', v

<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', v

<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', v

In [52]:
print(np.mean(lh_chan3))
print(np.mean(lh_chan4))
print(np.mean(lh_chan5))
print(np.mean(lh_chan6))
print('\n')
print(np.mean(lh_chan3_test))
print(np.mean(lh_chan4_test))
print(np.mean(lh_chan5_test))
print(np.mean(lh_chan6_test))

4.02791136227876
6.0754684314320295
5.192615659517384
6.029085672764467


3.97635083970883
6.585286500747188
5.051902468450313
12.355158968976644


In [53]:
####RH MI

rh_chan3 = bandpower_calc(chan_3_train[2],8,26)
rh_chan4 = bandpower_calc(chan_4_train[2],8,26)
rh_chan5 = bandpower_calc(chan_5_train[2],8,26)
rh_chan6 = bandpower_calc(chan_6_train[2],8,26)

rh_chan3_test = bandpower_calc(chan_3_test[2],8,26)
rh_chan4_test = bandpower_calc(chan_4_test[2],8,26)
rh_chan5_test = bandpower_calc(chan_5_test[2],8,26)
rh_chan6_test = bandpower_calc(chan_6_test[2],8,26)

<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', v

<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', v

<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', v

<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', v

In [54]:
print(np.mean(rh_chan3))
print(np.mean(rh_chan4))
print(np.mean(rh_chan5))
print(np.mean(rh_chan6))
print('\n')
print(np.mean(rh_chan3_test))
print(np.mean(rh_chan4_test))
print(np.mean(rh_chan5_test))
print(np.mean(rh_chan6_test))

4.1163411807675265
6.289089039990858
5.032605372149238
6.213927033462789


4.269558299474225
6.837734534801044
5.270854663513969
12.284292024092542


In [55]:
####TNG MI

tng_chan3 = bandpower_calc(chan_3_train[3],8,26)
tng_chan4 = bandpower_calc(chan_4_train[3],8,26)
tng_chan5 = bandpower_calc(chan_5_train[3],8,26)
tng_chan6 = bandpower_calc(chan_6_train[3],8,26)

tng_chan3_test = bandpower_calc(chan_3_test[3],8,26)
tng_chan4_test = bandpower_calc(chan_4_test[3],8,26)
tng_chan5_test = bandpower_calc(chan_5_test[3],8,26)
tng_chan6_test = bandpower_calc(chan_6_test[3],8,26)

<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', v

<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', v

<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', v

<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', verbose=0)
<ipython-input-46-34395bffbe27>:5: RuntimeWarning: Iterative multi-taper PSD computation did not converge.
  psd, freqs = psd_array_multitaper(trial, 250, adaptive=True,normalization='full', v

In [56]:
print(np.mean(tng_chan3))
print(np.mean(tng_chan4))
print(np.mean(tng_chan5))
print(np.mean(tng_chan6))
print('\n')
print(np.mean(tng_chan3_test))
print(np.mean(tng_chan4_test))
print(np.mean(tng_chan5_test))
print(np.mean(tng_chan6_test))

6.351437726025975
7.03176792022434
6.042956088686276
6.980494656726749


5.706486074606842
6.744405944237343
5.902559187068533
12.206361688182632


In [ ]:
####Non-converging Multi-taper PSD computation errors should be investigated####